In [46]:
import sys
import cantera as ct
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
import pandas as pd

import phases
import feedstock as fs
import fuel as fu
import energy as en
import gasifier as gs

In [47]:
coalUltimate = [78.4750, 3.9681, 16.0249, 0.7044, 0.7748, 0.0528] # % daf
coalAshWB = 0.128199 # fraction, w.b.
coalMoistWB = 0.02 # fraction, w.b.
coalHHV_WB = 24.727 # MJ/kg, w.b.
coalLHV_WB = 24.155 # MJ/kg, w.b.
ashMW = 80.7 # kg/kmol
ashHF = -788.92 # MJ/kmol
coalHF = -44.17 # MJ/kmol

coalHHV_DB = coalHHV_WB / (1 - coalMoistWB) # MJ/kg, d.b.
coalLHV_DB = coalLHV_WB / (1 - coalMoistWB) # MJ/kg, d.b.
coalAshDB = coalAshWB / (1 - coalMoistWB) # fraction, d.b.
coalMoistDB = coalMoistWB / (1 - coalMoistWB) # fraction, d.b.
ashComposition = [54.06, 6.57, 23.18, 6.85, 0.82, 1.6, 1.83, 0.5, 1.05, 3.54, 0] # % of ash
coal = fs.create_fuel_stream(10535, coalUltimate, coalAshDB, 5715/10535, HHV=coalHHV_DB, LHV=coalLHV_DB, ashComposition=ashComposition)
initialCarbon = coal.species_moles[phases.indices['C(gr)']] * phases.Mw['C(gr)']

In [48]:
O2 = fs.create_O2_stream(8750)
O2.T = 60 + 273.15 # K
coal.T = 60 + 273.15 # K
O2.P = 4.2E6 # Pa
coal.P = 4.2E6 # Pa

In [49]:
outlet = gs.gasify_isot(coal, O2, T=1274+273.15, P=4.2E6, charFormation=0.08004)
species = ['O2', 'CO', 'CO2', 'H2', 'H2O', 'CH4', 'N2']
soma = 0
for i, sp in enumerate(species):
    frac = outlet.get_syngas_fraction(sp)
    soma += frac
    print(f'{sp}: {frac}')
print(soma)
print(outlet.get_syngas_amount('mole', water=True, nitrogen=True))
print(outlet.species_moles[phases.indices['C(gr)']] * phases.Mw['C(gr)'] / initialCarbon)

O2: 3.384932125144767e-13
CO: 0.3781233220001839
CO2: 0.14585213511172707
H2: 0.22780175431599134
H2O: 0.24470406987725407
CH4: 7.984158280415671e-05
N2: 0.0021867323287846187
0.9987478552170836
1050.0458605696501
0.08005124086204925
